Let setup some dependency

In [1]:
import data_extractor

In [2]:
from nltk.tokenize import RegexpTokenizer

In [3]:
from gensim import corpora

In [4]:
import json

let's do things a little different. We will data that contained the document id used in gensim and the document path

In [5]:
documents = []

In [6]:
docs = []

In [7]:
stopwords = data_extractor.stop_words()

In [8]:
stopwords.extend(['tuan','kerajaan','tahun','pertanyaan',
                  'malaysia','menteri','negara', 'lisan', 'soalan', 'jawapan',
                 'kementerian', 'pertua', 'dewan', 'rakyat', 'minta', 'orang'
                 ])

In [9]:
index = [] # because we process a list of text anyway

In [10]:
tokenizer = RegexpTokenizer(r'(?:RM)*\d+(?:,|.\d+)*|\w+') # Spoiler this is to extract word without punctuation

Now process the documents

In [11]:
for doc_path in data_extractor.document_walker(path="docs/results/"):
    doc = data_extractor.extract_text_single(doc_path)
    
    text = tokenizer.tokenize(doc.lower())
    index.append((doc_path, doc, text))
    text = [ word for word in text if word not in stopwords ]
    docs.append(text)

In [12]:
docs[0]

['no',
 '24',
 'parlimen',
 'pemberitahu',
 'tarikh',
 'so',
 'alan',
 'dato',
 'mansor',
 'bin',
 'othman',
 'nibong',
 'tebal',
 '21',
 'mac',
 '2017',
 'selasa',
 'dato',
 'mansor',
 'bin',
 'othman',
 'mente',
 'ri',
 'pertanian',
 'industri',
 'asas',
 'tani',
 'menyatakan',
 'status',
 'pelaksanaan',
 'program',
 'agropreneur',
 'muda',
 'dibentangkan',
 'bajet',
 '2016',
 'dipertua',
 'pertanian',
 'lndustri',
 'asas',
 'tani',
 'melaksanakan',
 'program',
 'agropreneur',
 'muda',
 'melahir',
 'membangunkan',
 'belia',
 'sektor',
 'pertanian',
 'industri',
 'asas',
 'tani',
 'bantuan',
 'diberikan',
 'geran',
 'berbentuk',
 'barangan',
 'in',
 'kind',
 'contribution',
 'belia',
 'layak',
 'latihan',
 'i',
 'kursus',
 'teknikal',
 'pengurusan',
 'kewangan',
 'termasuk',
 'penyediaan',
 'kertas',
 'rancangan',
 'perniagaan',
 'status',
 'pelaksanaan',
 'program',
 'agropreneur',
 'muda',
 '2016',
 'berikut',
 '1',
 'mensasarkan',
 'serama1',
 '1,000',
 'belia',
 'peruntukan',
 'rm

In [13]:
index[0]

('docs/results/20170321__DR_JawabLisan/soalan-24.pdf',
 "SOALAN NO: 24 PARLIMEN MALAYSIA PEMBERITAHU PERTANYAAN DEWAN RAKYAT PERTANYAAN DARIPADA TARIKH SO ALAN : Lisan : Dato' Mansor bin Othman [ Nibong Tebal] : 21 Mac 2017 (Selasa) Dato' Mansor bin Othman minta MENTE RI PERTANIAN DAN INDUSTRI ASAS TANI menyatakan: status pelaksanaan Program Agropreneur Muda yang dibentangkan oleh Kerajaan dalam bajet 2016.  JAWAPAN Tuan Yang Dipertua, Kementerian Pertanian dan lndustri Asas Tani telah melaksanakan Program Agropreneur Muda bagi melahir dan membangunkan belia dalam sektor pertanian dan industri asas tani. Bantuan yang diberikan adalah geran berbentuk barangan (in-kind contribution) kepada belia layak serta latihan I kursus teknikal serta pengurusan kewangan termasuk penyediaan kertas rancangan perniagaan. Status pelaksanaan Program Agropreneur Muda bagi tahun 2016 adalah seperti berikut: 1. Mensasarkan serama1 1,000 belia dengan peruntukan RM20 juta; 11. Seramai 3,088 belia telah berdaf

Now let's make a corpus

In [14]:
dictionary = corpora.Dictionary(docs)

Let's save all the things 

In [15]:
json.dump(index, open("index.json", "w"))

In [16]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [17]:
corpora.MmCorpus.serialize("malaysian_parl_clean.mm", corpus)

In [18]:
dictionary.save("malaysian_parl_clean.dict")

Now create the LDA model

In [19]:
from gensim import models, similarities

In [20]:
corpus = corpora.MmCorpus('malaysian_parl_clean.mm')

In [21]:
model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)

let's test with one document

In [22]:
bow = model.id2word.doc2bow(index[20][2])

In [23]:
doc_topics, word_topics, phi_value = model.get_document_topics(corpus[20], per_word_topics=True)

In [24]:
 doc_topics

[(6, 0.50803470595517686),
 (29, 0.041442107295662285),
 (40, 0.064767851743149182),
 (45, 0.38325533500601089)]

In [25]:
model.show_topic(11)

[('pihak', 0.0055191717375115837),
 ('no', 0.0039548251645578972),
 ('bin', 0.0038799874374940077),
 ('2017', 0.0038083812476355093),
 ('menyatakan', 0.003766905130913949),
 ('china', 0.003615086512276891),
 ('tindakan', 0.0034225535267946388),
 ('2', 0.0034065562869656754),
 ('1', 0.0033755325015486473),
 ('syarikat', 0.0033727531568216408)]

In [26]:
index[5][1]

'NO. SOALAN: 41 PEMBERITAHUAN PERTANYAAN DEWAN RAKYAT PERTANYAAN DARIPADA TARIKH SO ALAN LISAN YB. DATO\' SRI ABDUL AZEEZ BIN ABDUL RAHIM [ BALING ] 21MAC2016 YB. Dato\' Sri Abdul Azeez Bin Abdul Rahim [ Baling ] minta MENTERI PELANCONGAN DAN KEBUDAY AAN menyatakan adakah pihak Kementerian bercadang untuk memansuhkan khidmat pemandu pelancong bagi lawatan pelajar sekolah dalam negara. lni disebabkan lawatan pelajar . sekolah telah dirancang oleh guru pengiring dan memandangkan kos pemandu pelancong agak tinggi dan tidak berfungsi ketika lawatan pelajar sekolah, hal ini pastinya akan membebankan pihak sekolah dan pengusaha bas sekolah ketika musim cuti sekolah. 1  JAWAPAN: DATO\' SERI MOHAMED NAZRI BIN ABDUL AZIZ MENTERI PELANCONGAN DAN KEBUDAYAAN Tuan Yang di-Pertua, Setakat ini Kementerian Pelancongan dan Kebudayaan Malaysia (KPK) tidak berhasrat memansuhkan khidmat pemandu pelancong di Negara ini yang bertindak sebagai duta kecil dalam memberi gambaran imej sebenar Malaysia sebagai d

In [27]:
model.print_topics()

[(16,
  '0.006*"no" + 0.006*"bin" + 0.005*"langkah" + 0.005*"tarikh" + 0.005*"2017" + 0.004*"datuk" + 0.004*"jumlah" + 0.004*"menyatakan" + 0.004*"berhormat" + 0.004*"bawah"'),
 (14,
  '0.009*"program" + 0.007*"kes" + 0.006*"bin" + 0.005*"negeri" + 0.005*"1" + 0.005*"tarikh" + 0.004*"undang" + 0.004*"kawasan" + 0.004*"dato" + 0.004*"menyatakan"'),
 (6,
  '0.007*"projek" + 0.007*"1" + 0.006*"menyatakan" + 0.006*"perkhidmatan" + 0.005*"program" + 0.004*"tarikh" + 0.004*"2" + 0.004*"a" + 0.004*"bin" + 0.004*"kesihatan"'),
 (43,
  '0.006*"no" + 0.006*"harga" + 0.006*"2017" + 0.005*"2" + 0.005*"tarikh" + 0.005*"pemberitahuan" + 0.005*"negeri" + 0.005*"2016" + 0.005*"1" + 0.004*"menyatakan"'),
 (46,
  '0.008*"1" + 0.006*"hutang" + 0.005*"kewangan" + 0.004*"pembangunan" + 0.004*"memastikan" + 0.004*"yb" + 0.004*"tarikh" + 0.004*"jumlah" + 0.003*"2016" + 0.003*"menyatakan"'),
 (21,
  '0.009*"negeri" + 0.008*"kanak" + 0.006*"keselamatan" + 0.005*"bawah" + 0.005*"2017" + 0.005*"projek" + 0.005*"